# A Birthday Present

In [ ]:
import coursierapi._
interp.repositories() +:= MavenRepository.of("https://jitpack.io")

In [ ]:
import $ivy.`com.github.dzufferey::libgcode:0.1-SNAPSHOT`
import libgcode.utils.Viewer
import libgcode._
import libgcode.extractor._
import libgcode.generator._
import libgcode.utils._
import scala.collection.mutable.ArrayBuffer

## Config

Grinding sandstone with a diamond bit.
Go slow with a big depth of cut

In [ ]:
// 6mm ball
val conf1 = new Config
conf1.safeHeight = 5.0
conf1.feed = 180
conf1.endmillDiameter = 6.0
conf1.depthOfCut = 2.0 // 18.0
conf1.finishingPass = 0.0
conf1.travelHeight = 1.0
conf1.climb = false
conf1.stepOver = 0.2

// 12mm flat
val conf2 = new Config
conf2.safeHeight = 5.0
conf2.feed = 180
conf2.endmillDiameter = 12.0
conf2.depthOfCut = 8.0
conf2.finishingPass = 0.0
conf2.travelHeight = 1.0
conf2.climb = false
conf1.stepOver = 0.2

## Part 1: Heart Shape

With a scale of 1, the heart is about `x` in `[-16,16]`, `y` in `[-16, 12]`, `z` in `[-1,0]`.

Different types of curve can be found at http://www.mathematische-basteleien.de/heart.htm

In [ ]:
class HeartSurface(xScale: Double, yScale: Double, zScale: Double) {
    import math._
    def x(u: Double, v: Double): Double = {
        val u2pi = 2 * Pi* u
        xScale * v * (12*sin(u2pi) - 4*sin(3*u2pi))
    }
    def y(u: Double, v: Double): Double = {
        val u2pi = 2 * Pi* u
        yScale * v * (13*cos(u2pi) - 5*cos(2*u2pi) - 2*cos(3*u2pi) - cos(4*u2pi))
    }
    def z(u: Double, v: Double): Double = {
        -zScale * pow(v, 6)
    }
    def apply(u: Double, v: Double): (Double, Double, Double) = (x(u,v), y(u,v), z(u,v))
}

In [ ]:
val xScale = 2
val yScale = 2
val zScale = 15
val uTics = 500
val vTics = 200

### Surface

In [ ]:
class Heart extends Program(conf1) {
    
    val h = new HeartSurface(xScale, yScale, zScale)
    
    val uRange = (0 to uTics).map( x => x.toDouble / uTics )
    
    val vRange = (0 to vTics).map( x => math.sqrt(x.toDouble / vTics) )
    
    val minDist = 0.2
    val xPlungeOffset = 10
    
    // ignore points that are too close to each other
    def keep(x0: Double, y0: Double, z0: Double,
             x1: Double, y1: Double, z1: Double) = {
        val dx = x0 - x1
        val dy = y0 - y1
        val dz = z0 - z1
        math.sqrt(dx*dx + dy*dy + dz*dz) >= minDist
    }
    
    def body = {
        val cmds = scala.collection.mutable.ArrayBuffer.empty[Command]
        val start = h(0, 0)
        var x0: Double = start._1
        var y0: Double = start._2
        var z0: Double = start._3
        cmds += G(0, X(x0 - xPlungeOffset), Y(y0))
        cmds += G(0, Z(z0 + conf1.travelHeight))
        cmds += G(0, X(x0), Y(y0), Z(z0))
        for (v <- vRange; u <- uRange) {
            val (x1, y1, z1) = h(u, v)
            if (keep(x0,y0,z0,x1,y1,z1)) {
                cmds += G(1, X(x1), Y(y1), Z(z1))
                x0 = x1
                y0 = y1
                z0 = z1
            }
        }
        cmds += G(0, Z(start._3 + conf1.travelHeight))
        cmds += G(0, X(start._1), Y(start._2))
        cmds.toSeq
    }

}

In [ ]:
val h = new Heart
//h.display //FIXME this is too much for the built-in viewer
h.save("heart.nc")

### Outline

In [ ]:
class HeartOutline extends Program(conf1) {
    val h = new HeartSurface(xScale, yScale, zScale)
    val uRange = (0 to uTics).map( x => x.toDouble / uTics )
    val yPlungeOffset = 10
    def body = {
        val cmds = scala.collection.mutable.ArrayBuffer.empty[Command]
        val (x0, y0, _) = h(0, 1)
        cmds += G(0, X(x0), Y(y0 - yPlungeOffset))
        cmds += G(0, Z(conf1.travelHeight))
        cmds += G(0, X(x0), Y(y0), Z(0))
        for (u <- uRange) {
            val (x, y, _) = h(u, 1)
            cmds += G(1, X(x), Y(y))
        }
        G(0, Z(conf1.travelHeight))
        cmds += G(0, X(x0), Y(y0))
        cmds.toSeq
    }
}

In [ ]:
val h = new HeartOutline
h.display
h.save("heart_outline.nc")

## Part 2: Hole for a Candle

In [ ]:
val candleDiameter = 40.0
val candleHeight = 15.0
val chamfer = 4.0
val chamferRadiusDelta = 2.0

### Roughing

In [ ]:
class HoleRoughing extends Program(conf1) {
    def body = {
        //Hole.roughing(0, 0, 0, candleDiameter/2, candleHeight, false)(conf)
        Hole.finishing(0, 0, 0, candleDiameter/2, candleHeight, false)(conf)
    }
}

In [ ]:
val h = new HoleRoughing
h.display
h.save("hole_roughing.nc")

### Finishing the hole

In [ ]:
class HoleFinishing extends Program(conf2) {
    def body = {
        Hole.finishing(0, 0, 0, candleDiameter/2, candleHeight, false)(conf)
    }
}

In [ ]:
val h = new HoleFinishing
h.display
h.save("hole_finishing.nc")

### Chamfering the Edges

In [ ]:
class HoleChamfer extends Program(conf1) {
    def body = {
        Spiral(0, 0, -(chamfer+chamferRadiusDelta), candleDiameter/2 - chamferRadiusDelta)(conf)
    }
}

In [ ]:
val h = new HoleChamfer
h.display
h.save("hole_chamfer.nc")